# パーセプトロン
このNotebookで、算数だけで単純なニューラルネットワーク（パーセプトロン）を構築し、「AND」と「OR」の論理演算を学習させてみよう

まず、必要なパッケージ（機能）を挿入しよう

In [1]:
# numpyはベクトル、または行列を管理しやすいパッケージである。
import numpy as np 

## numpyの操作
まず、numpy配列の操作を少し把握しないといけない。numpyでは、「行」と「列」を別々で処理できる。例えば、この行列

\begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \end{bmatrix}

numpyでこうなる：

In [2]:
test_matrix = np.array([[1, 2, 3], [4, 5, 6]])

# 2行、3列であることを確認！
print(test_matrix.shape)

(2, 3)


In [3]:
#中身も確認
print(test_matrix)

[[1 2 3]
 [4 5 6]]


In [4]:
#また、1個の要素だけを見たい場合、0から始まる指数でアクセスできる：
print(test_matrix[1, 0])

4


In [5]:
# 1行目を確認
print(test_matrix[0,:])

[1 2 3]


In [6]:
# 2列目を確認
print(test_matrix[:,1])

[2 5]


In [7]:
# もちろん、行か列単位で計算もOK
print(test_matrix[0,:] + test_matrix[1,:])

[5 7 9]


In [8]:
# また、数値・演算・numpy も可能
# 例えば、足し算
print (1 + test_matrix)

[[2 3 4]
 [5 6 7]]


In [9]:
# 乗算
print (2 * test_matrix[0,:])

[2 4 6]


## パーセプトロンの構築
つぎ、入力２つ、出力１つがあるパーセプトロンを構築しよう。何回も作るかもしれないので、関数で実装：

In [10]:
def create_perceptron():    
    model = {
        # 荷重を -5 ~ 5 の乱数で正気化
        "weights" : np.random.uniform(-5, 5, 2),    
        # バイアスも！
        "bias"    : np.random.uniform(-5, 5, 1)}

    return model

モデルを１つ作って、中身を確認

In [11]:
model = create_perceptron()
print(model)

{'weights': array([-2.48300611,  4.56714744]), 'bias': array([0.35598693])}


当然ながら、乱数はみんなが違うので、統一するため、乱数の「シード」（種）を決めましょう

In [12]:
np.random.seed(12345)

model = create_perceptron()
print(model)

{'weights': array([ 4.29616093, -1.83624445]), 'bias': array([-3.16081188])}


## 推定
パーセプトロンの推定は:

$$y^{pred}_j = \sum^n_{i=1} x_{ij}w_i + bias$$

今回ば、入力は２つのみで、xは「入力配列」であり、1行あたりは、1サンプル（学習データの1点）にする

In [13]:
def predict(model, activation, x):
    
    # 各サンプルの入力とその荷重の乗算を足して、最後のバイアスをいれて…
    y = model["weights"][0] * x[:,0] + model["weights"][1] * x[:,1] + model["bias"]
    
    # 活性化を返す
    y = activation(y)
    return y

このタイミングで、活性化関数はあまり気にしないで、とりあえず何もしない「線形関数」でいきましょう

In [14]:
def linear_activation(x):
    return x

入力データを用意し、推定してみましょう！

In [15]:
x = np.array([
    [0, 0], 
    [0, 1], 
    [1, 0], 
    [1, 1]
], dtype = float)

print(x.shape)

(4, 2)


In [16]:
# 推定
y_pred = predict(model, linear_activation, x)
print(y_pred.shape)

(4,)


In [17]:
# 各入力データポイントの結果
print(y_pred)

[-3.16081188 -4.99705634  1.13534904 -0.70089541]


## 学習
当たり前が、まだ学習されていないので、推定はごみ。学習のため、損失関数、損失関数の勾配、そして荷重変化は以下の通りに（細かい説明はスライドで）

$$loss(W) = \frac{1}{2m}\sum_{j=1}^{m}(y^{true}_j-y^{pred}_j)^2$$
$$\nabla{loss(W)} = \Delta{W} = -\frac{1}{m}\sum_{j=1}^{m}(y^{true}_j-y^{pred}_j)\cdot X_j$$
$$W'=W + \Delta{W} \cdot lr$$


In [18]:
# 損失関数は'mean squared error'（平均二乗誤差）とする
def mean_squared_error(y_true, y_pred):
    error = y_true - y_pred
    mse = np.mean(error * error)
    return mse

In [19]:
# y_trueを定義しないと！ANDの場合は：
y_true = np.array([ 0, 0, 0, 1 ], dtype=float)

In [20]:
# さて、推定と実際のデータの損失は？
print(y_true)
print(y_pred)
print(mean_squared_error(y_true, y_pred))

[0. 0. 0. 1.]
[-3.16081188 -4.99705634  1.13534904 -0.70089541]
9.785841611861958


In [21]:
# 次、勾配関数を作ろう～
def mse_gradient(x, y_true, y_pred):
    
    error = y_true - y_pred

    grad_w = - x * error
    grad_b = - 1 * error

    return np.mean(grad_w, axis=0), np.mean(grad_b, axis=0)

In [22]:
# 勾配（荷重変化）はこんな感じ：
print (mse_gradient(x, y_true, y_pred))

ValueError: operands could not be broadcast together with shapes (4,2) (4,) 

あれ？なにこのエラー？

`ValueError: operands could not be broadcast together with shapes (4,2) (4,) `

なんか、shapeが合わないっぽい！

In [23]:
# shapeを確認すると：
error = y_true - y_pred
print(x.shape)
print(error.shape)

(4, 2)
(4,)


In [24]:
# なるほど。errorの「列」固定ではない（サイズ変えてもOKのShape）
# reshapeで固定しましょう
error = error.reshape(4, 1)
print(error.shape)

(4, 1)


In [25]:
#これならば, 乗算できるよね？
print(-x * error)

[[-0.         -0.        ]
 [-0.         -4.99705634]
 [ 1.13534904  0.        ]
 [-1.70089541 -1.70089541]]


In [26]:
# で、勾配関数を直そう～
def mse_gradient(x, y_true, y_pred):
    
    error = y_true - y_pred

    # errorのshapeがxのshapeが合わないため、reshapeが必要
    error = error.reshape(x.shape[0], 1)
    
    grad_w = - x * error
    grad_b = - 1 * error

    return np.mean(grad_w, axis=0), np.mean(grad_b, axis=0)

In [27]:
# 勾配（荷重変化）はこんな感じ：
print (mse_gradient(x, y_true, y_pred))

(array([-0.14138659, -1.67448794]), array([-2.18085365]))


それでは、必要な関数ができたので、学習関数を作ろう！

In [28]:
# 学習関数
def fit(model, activation, loss, gradient, learning_rate, x, y_true):
    
    # 推定
    y_pred = predict(model, activation, x)
    
    # 損失
    print("Loss:", loss(y_true, y_pred))
    
    # 勾配
    grad_w, grad_b = gradient(x, y_true, y_pred)
    
    # 荷重調整
    adjust_w = grad_w * learning_rate
    adjust_b = grad_b * learning_rate
    
    # モデル更新
    model["weights"] = model["weights"] - adjust_w
    model["bias"]    = model["bias"]    - adjust_b

In [29]:
# 試してみましょう！

print (y_true)

# before
y_pred = predict(model, linear_activation, x)
print (y_pred)

fit(model, linear_activation, mean_squared_error, mse_gradient, 0.01, x, y_true)

# after
y_pred = predict(model, linear_activation, x)
print (y_pred)

[0. 0. 0. 1.]
[-3.16081188 -4.99705634  1.13534904 -0.70089541]
Loss: 9.785841611861958
[-3.13900335 -4.95850292  1.15857145 -0.66092813]


In [30]:
# 同じことを何回も学習させる（epochs)
def fit_epochs(model, activation, loss, gradient, learning_rate, x, y_true, epochs):
    
    for i in range(epochs):
        fit(model, activation, loss, gradient, learning_rate, x, y_true)

In [31]:
fit_epochs(model, linear_activation, mean_squared_error, mse_gradient, 0.01, x, y_true, 3000)

Loss: 9.635265819401875
Loss: 9.488725362135831
Loss: 9.346096123854624
Loss: 9.20725789481097
Loss: 9.072094248326433
Loss: 8.940492421298066
Loss: 8.812343198481594
Loss: 8.687540800431716
Loss: 8.565982774983963
Loss: 8.447569892166161
Loss: 8.332206042431118
Loss: 8.21979813810552
Loss: 8.110256017953434
Loss: 8.00349235475594
Loss: 7.899422565811563
Loss: 7.79796472626521
Loss: 7.699039485176176
Loss: 7.602569984238688
Loss: 7.508481779071096
Loss: 7.416702762992597
Loss: 7.327163093208776
Loss: 7.23979511932992
Loss: 7.154533314148306
Loss: 7.071314206603104
Loss: 6.990076316863726
Loss: 6.910760093464696
Loss: 6.833307852427144
Loss: 6.757663718304205
Loss: 6.683773567089455
Loss: 6.611584970929524
Loss: 6.541047144583863
Loss: 6.472110893576448
Loss: 6.404728563985925
Loss: 6.338853993822423
Loss: 6.2744424659409
Loss: 6.211450662442422
Loss: 6.149836620516381
Loss: 6.089559689678096
Loss: 6.03058049035769
Loss: 5.972860873797535
Loss: 5.916363883216912
Loss: 5.861053716203832


Loss: 0.4818563610765316
Loss: 0.4800858433246198
Loss: 0.4783231521262094
Loss: 0.4765682515799741
Loss: 0.4748211059539063
Loss: 0.4730816796845027
Loss: 0.4713499373759527
Loss: 0.4696258437993324
Loss: 0.46790936389180143
Loss: 0.4662004627558031
Loss: 0.4644991056582707
Loss: 0.4628052580298353
Loss: 0.4611188854640387
Loss: 0.45943995371654994
Loss: 0.45776842870438617
Loss: 0.4561042765051356
Loss: 0.4544474633561865
Loss: 0.452797955653959
Loss: 0.45115571995313963
Loss: 0.4495207229659213
Loss: 0.4478929315612463
Loss: 0.4462723127640524
Loss: 0.4446588337545237
Loss: 0.4430524618673441
Loss: 0.4414531645909554
Loss: 0.4398609095668185
Loss: 0.43827566458867767
Loss: 0.43669739760182996
Loss: 0.43512607670239634
Loss: 0.4335616701365979
Loss: 0.4320041463000344
Loss: 0.4304534737369675
Loss: 0.4289096211396063
Loss: 0.4273725573473974
Loss: 0.42584225134631765
Loss: 0.42431867226817094
Loss: 0.42280178938988816
Loss: 0.42129157213283014
Loss: 0.4197879900620949
Loss: 0.4182910

Loss: 0.08155921144918515
Loss: 0.08149084474582566
Loss: 0.08142273536829396
Loss: 0.08135488229525628
Loss: 0.08128728450965227
Loss: 0.08121994099867608
Loss: 0.08115285075375776
Loss: 0.08108601277054478
Loss: 0.08101942604888343
Loss: 0.08095308959280044
Loss: 0.0808870024104847
Loss: 0.0808211635142689
Loss: 0.08075557192061154
Loss: 0.08069022665007876
Loss: 0.08062512672732633
Loss: 0.08056027118108176
Loss: 0.08049565904412657
Loss: 0.08043128935327835
Loss: 0.08036716114937331
Loss: 0.0803032734772485
Loss: 0.0802396253857244
Loss: 0.08017621592758756
Loss: 0.08011304415957306
Loss: 0.08005010914234748
Loss: 0.07998740994049149
Loss: 0.07992494562248294
Loss: 0.07986271526067963
Loss: 0.07980071793130251
Loss: 0.07973895271441875
Loss: 0.07967741869392488
Loss: 0.07961611495753017
Loss: 0.07955504059673989
Loss: 0.07949419470683879
Loss: 0.07943357638687458
Loss: 0.0793731847396415
Loss: 0.079313018871664
Loss: 0.07925307789318045
Loss: 0.07919336091812688
Loss: 0.07913386706

Loss: 0.06676212488779258
Loss: 0.06674776485839849
Loss: 0.06673345482496787
Loss: 0.06671919460593267
Loss: 0.066704984020418
Loss: 0.06669082288823931
Loss: 0.06667671102989958
Loss: 0.06666264826658663
Loss: 0.06664863442017033
Loss: 0.06663466931319974
Loss: 0.06662075276890067
Loss: 0.06660688461117258
Loss: 0.06659306466458623
Loss: 0.06657929275438068
Loss: 0.06656556870646085
Loss: 0.06655189234739464
Loss: 0.0665382635044104
Loss: 0.06652468200539424
Loss: 0.06651114767888737
Loss: 0.0664976603540835
Loss: 0.06648421986082616
Loss: 0.06647082602960622
Loss: 0.06645747869155913
Loss: 0.06644417767846242
Loss: 0.06643092282273315
Loss: 0.0664177139574253
Loss: 0.06640455091622721
Loss: 0.0663914335334591
Loss: 0.06637836164407042
Loss: 0.06636533508363746
Loss: 0.06635235368836082
Loss: 0.06633941729506274
Loss: 0.06632652574118482
Loss: 0.06631367886478551
Loss: 0.06630087650453753
Loss: 0.06628811849972546
Loss: 0.06627540469024339
Loss: 0.06626273491659226
Loss: 0.0662501090

Loss: 0.06333488356122209
Loss: 0.0633321814860184
Loss: 0.06332948830320832
Loss: 0.06332680398281512
Loss: 0.06332412849496646
Loss: 0.06332146180989415
Loss: 0.06331880389793354
Loss: 0.06331615472952343
Loss: 0.06331351427520544
Loss: 0.06331088250562387
Loss: 0.06330825939152512
Loss: 0.06330564490375742
Loss: 0.06330303901327046
Loss: 0.06330044169111503
Loss: 0.06329785290844253
Loss: 0.06329527263650488
Loss: 0.06329270084665384
Loss: 0.0632901375103408
Loss: 0.06328758259911647
Loss: 0.06328503608463046
Loss: 0.06328249793863082
Loss: 0.0632799681329639
Loss: 0.06327744663957384
Loss: 0.06327493343050218
Loss: 0.0632724284778877
Loss: 0.06326993175396586
Loss: 0.06326744323106856
Loss: 0.06326496288162381
Loss: 0.06326249067815524
Loss: 0.063260026593282
Loss: 0.06325757059971815
Loss: 0.0632551226702724
Loss: 0.063252682777848
Loss: 0.063250250895442
Loss: 0.06324782699614524
Loss: 0.06324541105314177
Loss: 0.06324300303970876
Loss: 0.06324060292921588
Loss: 0.063238210695125

Loss: 0.06279435467159267
Loss: 0.06279341499224567
Loss: 0.06279247834247562
Loss: 0.06279154471236881
Loss: 0.06279061409204467
Loss: 0.06278968647165566
Loss: 0.06278876184138711
Loss: 0.06278784019145728
Loss: 0.06278692151211698
Loss: 0.06278600579364976
Loss: 0.06278509302637153
Loss: 0.06278418320063053
Loss: 0.06278327630680741
Loss: 0.06278237233531478
Loss: 0.06278147127659739
Loss: 0.06278057312113187
Loss: 0.06277967785942665
Loss: 0.06277878548202181
Loss: 0.06277789597948918
Loss: 0.06277700934243188
Loss: 0.06277612556148453
Loss: 0.06277524462731292
Loss: 0.06277436653061408
Loss: 0.06277349126211604
Loss: 0.06277261881257779
Loss: 0.06277174917278913
Loss: 0.06277088233357064
Loss: 0.06277001828577344
Loss: 0.0627691570202793
Loss: 0.06276829852800028
Loss: 0.06276744279987878
Loss: 0.06276658982688743
Loss: 0.06276573960002899
Loss: 0.06276489211033615
Loss: 0.0627640473488715
Loss: 0.06276320530672753
Loss: 0.06276236597502628
Loss: 0.06276152934491946
Loss: 0.062760

In [32]:
# 試してみましょう！

print (y_true)
y_pred = predict(model, linear_activation, x)
print (y_pred)

[0. 0. 0. 1.]
[-0.26535257  0.24591491  0.24926654  0.76053402]


## 適する活性化関数

### ステップ関数

$$ step(x)=\begin{cases} 1 & x\geq 0.5 \text{ の場合}\\ \\ 0 & \text{以外} \end{cases}$$

<img src="step.png" alt="step function" style="width: 650px;"/>

ただ、ステップ関数は微分ができないので、正確に学習できない。それに近い「シグモイド関数」を使おう：

### シグモイド関数

$$ sigmod(x) = \frac{1}{1 + e^{-x}} $$

<img src="sigmoid.png" alt="sigmoid function" style="width: 650px;"/>



In [33]:
def sigmoid_activation(x):
    return 1 / (1 + np.power(np.e, -x))

In [34]:
# モデルを再構築し、シグモイド関数の活性化で学習させよ！
np.random.seed(12345)
model = create_perceptron()

In [35]:
fit_epochs(model, sigmoid_activation, mean_squared_error, mse_gradient, 0.01, x, y_true, 5000)

Loss: 0.2553056778192273
Loss: 0.25518538826963366
Loss: 0.25506514190938345
Loss: 0.2549449387449596
Loss: 0.25482477878252324
Loss: 0.25470466202791603
Loss: 0.2545845884866628
Loss: 0.2544645581639743
Loss: 0.25434457106474984
Loss: 0.25422462719357974
Loss: 0.2541047265547483
Loss: 0.25398486915223595
Loss: 0.253865054989722
Loss: 0.2537452840705875
Loss: 0.2536255563979172
Loss: 0.2535058719745026
Loss: 0.2533862308028445
Loss: 0.2532666328851551
Loss: 0.25314707822336097
Loss: 0.2530275668191052
Loss: 0.25290809867375
Loss: 0.25278867378837955
Loss: 0.25266929216380174
Loss: 0.25254995380055134
Loss: 0.25243065869889203
Loss: 0.25231140685881903
Loss: 0.2521921982800613
Loss: 0.2520730329620843
Loss: 0.251953910904092
Loss: 0.25183483210502966
Loss: 0.251715796563586
Loss: 0.25159680427819553
Loss: 0.25147785524704086
Loss: 0.25135894946805537
Loss: 0.25124008693892536
Loss: 0.2511212676570922
Loss: 0.2510024916197549
Loss: 0.25088375882387237
Loss: 0.2507650692661655
Loss: 0.250

Loss: 0.21387777775856043
Loss: 0.2137726027422186
Loss: 0.21366746594178632
Loss: 0.21356236734609035
Loss: 0.21345730694402187
Loss: 0.2133522847245362
Loss: 0.21324730067665335
Loss: 0.2131423547894573
Loss: 0.21303744705209698
Loss: 0.21293257745378538
Loss: 0.2128277459838001
Loss: 0.212722952631483
Loss: 0.21261819738624055
Loss: 0.21251348023754368
Loss: 0.21240880117492772
Loss: 0.21230416018799253
Loss: 0.21219955726640238
Loss: 0.212094992399886
Loss: 0.21199046557823664
Loss: 0.21188597679131171
Loss: 0.21178152602903347
Loss: 0.21167711328138822
Loss: 0.21157273853842687
Loss: 0.21146840179026455
Loss: 0.21136410302708092
Loss: 0.2112598422391198
Loss: 0.21115561941668948
Loss: 0.21105143455016226
Loss: 0.2109472876299749
Loss: 0.21084317864662847
Loss: 0.2107391075906879
Loss: 0.21063507445278246
Loss: 0.21053107922360553
Loss: 0.21042712189391455
Loss: 0.2103232024545309
Loss: 0.21021932089634
Loss: 0.21011547721029117
Loss: 0.21001167138739762
Loss: 0.2099079034187365
Lo

Loss: 0.16605246678313135
Loss: 0.16596600694716857
Loss: 0.1658795851035755
Loss: 0.16579320125544927
Loss: 0.16570685540587496
Loss: 0.16562054755792593
Loss: 0.16553427771466364
Loss: 0.16544804587913756
Loss: 0.1653618520543848
Loss: 0.16527569624343053
Loss: 0.16518957844928744
Loss: 0.1651034986749556
Loss: 0.1650174569234229
Loss: 0.16493145319766422
Loss: 0.16484548750064196
Loss: 0.16475955983530555
Loss: 0.1646736702045914
Loss: 0.1645878186114231
Loss: 0.16450200505871093
Loss: 0.16441622954935187
Loss: 0.16433049208622985
Loss: 0.1642447926722149
Loss: 0.16415913131016396
Loss: 0.16407350800292014
Loss: 0.16398792275331278
Loss: 0.16390237556415754
Loss: 0.16381686643825616
Loss: 0.16373139537839607
Loss: 0.16364596238735113
Loss: 0.16356056746788045
Loss: 0.16347521062272924
Loss: 0.16338989185462818
Loss: 0.1633046111662935
Loss: 0.1632193685604269
Loss: 0.16313416403971523
Loss: 0.1630489976068309
Loss: 0.1629638692644313
Loss: 0.16287877901515896
Loss: 0.162793726861641

Loss: 0.1275783481673305
Loss: 0.12751086518493243
Loss: 0.1274434191479271
Loss: 0.12737601004835342
Loss: 0.12730863787822905
Loss: 0.1272413026295502
Loss: 0.1271740042942919
Loss: 0.12710674286440768
Loss: 0.12703951833183005
Loss: 0.1269723306884702
Loss: 0.12690517992621808
Loss: 0.12683806603694264
Loss: 0.1267709890124915
Loss: 0.12670394884469133
Loss: 0.12663694552534754
Loss: 0.12656997904624487
Loss: 0.12650304939914664
Loss: 0.12643615657579546
Loss: 0.12636930056791287
Loss: 0.12630248136719957
Loss: 0.12623569896533535
Loss: 0.12616895335397912
Loss: 0.12610224452476895
Loss: 0.1260355724693223
Loss: 0.1259689371792356
Loss: 0.12590233864608472
Loss: 0.12583577686142483
Loss: 0.12576925181679038
Loss: 0.12570276350369522
Loss: 0.12563631191363261
Loss: 0.12556989703807522
Loss: 0.12550351886847513
Loss: 0.12543717739626403
Loss: 0.12537087261285307
Loss: 0.125304604509633
Loss: 0.12523837307797397
Loss: 0.12517217830922608
Loss: 0.12510602019471878
Loss: 0.12503989872576

Loss: 0.0982848460186175
Loss: 0.0982344554762612
Loss: 0.09818409594770841
Loss: 0.09813376741854249
Loss: 0.09808346987434179
Loss: 0.09803320330067991
Loss: 0.09798296768312559
Loss: 0.09793276300724288
Loss: 0.09788258925859106
Loss: 0.09783244642272455
Loss: 0.09778233448519326
Loss: 0.09773225343154233
Loss: 0.0976822032473123
Loss: 0.09763218391803906
Loss: 0.09758219542925387
Loss: 0.09753223776648354
Loss: 0.09748231091525028
Loss: 0.09743241486107185
Loss: 0.09738254958946146
Loss: 0.09733271508592788
Loss: 0.09728291133597547
Loss: 0.09723313832510422
Loss: 0.09718339603880968
Loss: 0.09713368446258312
Loss: 0.09708400358191147
Loss: 0.09703435338227731
Loss: 0.09698473384915908
Loss: 0.09693514496803077
Loss: 0.09688558672436247
Loss: 0.09683605910361975
Loss: 0.0967865620912643
Loss: 0.09673709567275343
Loss: 0.09668765983354054
Loss: 0.09663825455907488
Loss: 0.0965888798348016
Loss: 0.09653953564616187
Loss: 0.09649022197859283
Loss: 0.09644093881752763
Loss: 0.096391686

Loss: 0.07665364015777419
Loss: 0.07661691765864992
Loss: 0.07658021882358874
Loss: 0.07654354363834676
Loss: 0.07650689208868383
Loss: 0.07647026416036407
Loss: 0.07643365983915562
Loss: 0.0763970791108306
Loss: 0.07636052196116536
Loss: 0.07632398837594023
Loss: 0.0762874783409397
Loss: 0.0762509918419523
Loss: 0.07621452886477081
Loss: 0.07617808939519205
Loss: 0.07614167341901698
Loss: 0.07610528092205072
Loss: 0.07606891189010259
Loss: 0.07603256630898603
Loss: 0.07599624416451864
Loss: 0.07595994544252224
Loss: 0.07592367012882283
Loss: 0.07588741820925071
Loss: 0.07585118966964019
Loss: 0.07581498449583002
Loss: 0.07577880267366296
Loss: 0.07574264418898623
Loss: 0.07570650902765111
Loss: 0.07567039717551333
Loss: 0.07563430861843265
Loss: 0.07559824334227327
Loss: 0.07556220133290367
Loss: 0.07552618257619653
Loss: 0.07549018705802893
Loss: 0.07545421476428218
Loss: 0.07541826568084194
Loss: 0.07538233979359821
Loss: 0.0753464370884453
Loss: 0.07531055755128184
Loss: 0.07527470

Loss: 0.049599893086115426
Loss: 0.04958060912379233
Loss: 0.04956133734525937
Loss: 0.049542077741986476
Loss: 0.04952283030544917
Loss: 0.04950359502712843
Loss: 0.04948437189851078
Loss: 0.04946516091108835
Loss: 0.049445962056358705
Loss: 0.049426775325824984
Loss: 0.04940760071099588
Loss: 0.04938843820338556
Loss: 0.04936928779451366
Loss: 0.04935014947590545
Loss: 0.04933102323909169
Loss: 0.04931190907560862
Loss: 0.04929280697699792
Loss: 0.04927371693480689
Loss: 0.049254638940588305
Loss: 0.04923557298590048
Loss: 0.04921651906230712
Loss: 0.04919747716137754
Loss: 0.049178447274686454
Loss: 0.049159429393814194
Loss: 0.04914042351034649
Loss: 0.04912142961587459
Loss: 0.0491024477019952
Loss: 0.049083477760310565
Loss: 0.049064519782428366
Loss: 0.0490455737599618
Loss: 0.04902663968452953
Loss: 0.04900771754775564
Loss: 0.04898880734126981
Loss: 0.0489699090567071
Loss: 0.048951022685708015
Loss: 0.048932148219918606
Loss: 0.04891328565099033
Loss: 0.048894434970580145
Los

Loss: 0.03520384267387328
Loss: 0.03519329303969243
Loss: 0.035182749415093924
Loss: 0.03517221179583716
Loss: 0.0351616801776847
Loss: 0.035151154556402144
Loss: 0.03514063492775825
Loss: 0.03513012128752482
Loss: 0.035119613631476705
Loss: 0.035109111955391924
Loss: 0.035098616255051576
Loss: 0.03508812652623977
Loss: 0.035077642764743774
Loss: 0.03506716496635387
Loss: 0.03505669312686347
Loss: 0.03504622724206905
Loss: 0.035035767307770124
Loss: 0.035025313319769324
Loss: 0.03501486527387228
Loss: 0.035004423165887796
Loss: 0.034993986991627604
Loss: 0.034983556746906674
Loss: 0.034973132427542825
Loss: 0.034962714029357106
Loss: 0.03495230154817354
Loss: 0.0349418949798192
Loss: 0.03493149432012427
Loss: 0.034921099564921926
Loss: 0.03491071071004843
Loss: 0.034900327751343024
Loss: 0.03488995068464806
Loss: 0.03487957950580895
Loss: 0.03486921421067404
Loss: 0.034858854795094804
Loss: 0.03484850125492575
Loss: 0.03483815358602437
Loss: 0.03482781178425115
Loss: 0.0348174758454697

In [36]:
print(predict(model, sigmoid_activation, x))

[0.00881682 0.08585146 0.23469099 0.76402128]


In [37]:
np.set_printoptions(suppress=True)
print(predict(model, sigmoid_activation, x))

[0.00881682 0.08585146 0.23469099 0.76402128]


## 練習:「OR」を学習させる